In [1]:
import json
import pandas as pd


In [2]:
# Load the JSON file
with open('./Spotify Account Data/StreamingHistory_music_0.json', 'r',encoding='utf-8') as file:
    data = json.load(file)

In [3]:
df = pd.DataFrame(data)
df.head()

,endTime,artistName,trackName,msPlayed
0,2023-08-12 12:42,Javed Ali,Ek Din Teri Raahon,231549
1,2023-08-17 15:54,Shoffy,I Can't Help,6490
2,2023-08-17 15:54,Sandesh Shandilya,Deewana Hai Dekho,20700
3,2023-08-17 15:54,Alka Yagnik,Tere Naam,2900
4,2023-08-17 15:57,Baby Keem,16,156040


In [5]:
df['minutesPlayed'] = df['msPlayed']/1000/60
df.head()

,endTime,artistName,trackName,msPlayed,minutesPlayed
0,2023-08-12 12:42,Javed Ali,Ek Din Teri Raahon,231549,3.859150
1,2023-08-17 15:54,Shoffy,I Can't Help,6490,0.108167
2,2023-08-17 15:54,Sandesh Shandilya,Deewana Hai Dekho,20700,0.345000
3,2023-08-17 15:54,Alka Yagnik,Tere Naam,2900,0.048333
4,2023-08-17 15:57,Baby Keem,16,156040,2.600667


In [15]:
df['endTime'] = pd.to_datetime(df['endTime'],format='%Y-%m-%d %H:%M')

In [16]:
df.describe()

,endTime,msPlayed,minutesPlayed
count,8370,8.370000e+03,8370.000000
mean,2024-02-28 05:16:50.473118464,1.657387e+05,2.762312
min,2023-08-12 12:42:00,0.000000e+00,0.000000
25%,2023-12-16 06:46:00,4.351000e+04,0.725167
50%,2024-03-02 11:03:30,1.881075e+05,3.135125
75%,2024-05-29 12:40:15,2.529870e+05,4.216450
max,2024-08-13 12:51:00,1.744536e+06,29.075600
std,NaN,1.137720e+05,1.896200


In [28]:
df[df['endTime']>='2024-01-01']['minutesPlayed'].sum()

14254.068533333333

In [29]:
df['year-month'] = df['endTime'].dt.to_period('M')
df.head()

,endTime,artistName,trackName,msPlayed,minutesPlayed,year-month
0,2023-08-12 12:42:00,Javed Ali,Ek Din Teri Raahon,231549,3.859150,2023-08
1,2023-08-17 15:54:00,Shoffy,I Can't Help,6490,0.108167,2023-08
2,2023-08-17 15:54:00,Sandesh Shandilya,Deewana Hai Dekho,20700,0.345000,2023-08
3,2023-08-17 15:54:00,Alka Yagnik,Tere Naam,2900,0.048333,2023-08
4,2023-08-17 15:57:00,Baby Keem,16,156040,2.600667,2023-08


In [31]:
# # Convert 'msPlayed' from milliseconds to minutes
# df['minutesPlayed'] = df['msPlayed'] / 1000 / 60

# Group by 'year_month' and calculate the total minutes played per month
monthly_minutes = df.groupby('year-month')['minutesPlayed'].sum().reset_index()
monthly_minutes

,year-month,minutesPlayed
0,2023-08,586.904733
1,2023-09,369.773067
2,2023-10,1624.239750
3,2023-11,1889.132467
4,2023-12,4396.430150
5,2024-01,1321.780583
6,2024-02,999.316667
7,2024-03,1726.361500
8,2024-04,2411.234050
9,2024-05,2023.017517


In [34]:
# Calculate the average listening time per month
average_minutes_per_month = monthly_minutes['minutesPlayed'].mean()

print(f"Average minutes listened per month: {average_minutes_per_month:.2f} minutes or {average_minutes_per_month/60:.2f} hours")

Average minutes listened per month: 1778.50 minutes or 29.64 hours


In [42]:
# Group by year_month and artistName to sum up the minutes played for each artist each month
monthly_artist_minutes = df.groupby(['year-month', 'artistName'])['minutesPlayed'].sum().reset_index()

In [41]:
# Sort by year-month and minutesPlayed in descending order
monthly_artist_minutes = monthly_artist_minutes.sort_values(['year-month', 'minutesPlayed'], ascending=[True, False])

# Drop duplicates, keeping the first (which is the top artist for that month)
top_artists_each_month = monthly_artist_minutes.drop_duplicates(subset=['year-month'], keep='first').reset_index(drop=True)

print("Top artists for each month:")
print(top_artists_each_month)

Top artists for each month:
   year-month      artistName  minutesPlayed
0     2023-08      AP Dhillon      84.545617
1     2023-09  Vishal-Shekhar      25.493233
2     2023-10          Pritam     150.241983
3     2023-11          Pritam     300.500717
4     2023-12          Pritam     363.047417
5     2024-01    Arijit Singh      54.819500
6     2024-02          Pritam     105.389100
7     2024-03          Pritam     179.009133
8     2024-04          Pritam     225.650683
9     2024-05          Pritam     390.897117
10    2024-06          Pritam     433.436983
11    2024-07          Pritam     375.210667
12    2024-08          Pritam     125.624300


In [49]:
# Group by year-month and trackName to calculate the total minutes played and the count of plays for each track each month
monthly_track_stats = df.groupby(['year-month', 'trackName']).agg(
    total_minutes_played=('minutesPlayed', 'sum'),
    play_count=('trackName', 'size')
).reset_index()

# Sort by year-month and total_minutes_played in descending order
monthly_track_stats = monthly_track_stats.sort_values(['year-month', 'total_minutes_played'], ascending=[True, False])

# Drop duplicates, keeping the first (which is the top track for that month)
top_tracks_each_month = monthly_track_stats.drop_duplicates(subset=['year-month'], keep='first').reset_index(drop=True)

print("Top tracks for each month with their play count:")
top_tracks_each_month.head(15)

Top tracks for each month with their play count:


,year-month,trackName,total_minutes_played,play_count
0,2023-08,Dil Nu,43.979883,13
1,2023-09,Day Dreaming,7.040867,2
2,2023-10,Oh Ho Ho Ho (Remix),24.441600,6
3,2023-11,Saadi Galli Aaja,46.968883,15
4,2023-12,Borderline,38.551267,11
5,2024-01,Sooraj Ki Baahon Mein,21.448517,7
6,2024-02,Subhanallah,20.808917,5
7,2024-03,Chand Sifarish,22.710433,5
8,2024-04,Aahista Aahista,25.914517,7
9,2024-05,Main Kya Hoon,22.406767,5
